In [ ]:
!pip install azure-ai-textanalytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Initialize Azure Text Analytics client
endpoint = "https://200573393.cognitiveservices.azure.com/"
key = "1e9a05ac6f6240d9b2a67b8359485421"
text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Read your dataset into a Pandas DataFrame
df = pd.read_csv('customer_review.csv')  # Update the filename with your actual dataset

# Function to analyze sentiment for each chunk of text
def analyze_sentiment_chunks(text):
    sentiments = []
    confidence_scores = []
    # Split text into chunks of 5,000 characters (to stay below the limit)
    chunk_size = 5000
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i + chunk_size]
        language_result = text_analytics_client.detect_language(documents=[chunk])
        language = language_result[0].primary_language.iso6391_name
        response = text_analytics_client.analyze_sentiment(documents=[chunk], language=language)
        sentiments.append(response[0].sentiment)
        confidence_scores.append(response[0].confidence_scores)
    return sentiments, confidence_scores

# Apply sentiment analysis function to each row in DataFrame
sentiments = []
confidence_scores = []

for text in df['sentiment']:
    sentiment_chunks, scores_chunks = analyze_sentiment_chunks(text)
    sentiments.append(sentiment_chunks)
    confidence_scores.append(scores_chunks)

df['sentiment'] = sentiments
df['confidence_scores'] = confidence_scores

# Print DataFrame with sentiment analysis results
print(df)
